In [1]:
#Imports and Configuration
import logging
from dotenv import load_dotenv
import os
import boto3
from rich import print as rprint
from typing import List, Dict, Any
from botocore.exceptions import NoCredentialsError, ClientError

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the environment variables from .env file
load_dotenv()

#  database ID
table_name: str = os.getenv('TABLE_NAME_1')

# Initialize clients
try:
    dynamodb = boto3.client('dynamodb')
    dynamodbres = boto3.resource('dynamodb')
except NoCredentialsError as e:
    logging.error("Credentials not available for AWS or Notion Client: %s", e)
    raise
except Exception as e:
    logging.error("Error initializing clients: %s", e)
    raise

logging.info(f'Table name from environment variable: {table_name}')


2024-06-05 20:22:43,900 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-06-05 20:22:44,575 - INFO - Table name from environment variable: DynamoDB_Quick_Photos


In [2]:
import boto3

dynamodb = boto3.client('dynamodb')

try:
    dynamodb.create_table(
        TableName='quick-photos',
        AttributeDefinitions=[
            {
                "AttributeName": "PK",
                "AttributeType": "S"
            },
            {
                "AttributeName": "SK",
                "AttributeType": "S"
            }
        ],
        KeySchema=[
            {
                "AttributeName": "PK",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "SK",
                "KeyType": "RANGE"
            }
        ],
        ProvisionedThroughput={
            "ReadCapacityUnits": 5,
            "WriteCapacityUnits": 5
        }
    )
    print("Table created successfully.")
except Exception as e:
    print("Could not create table. Error:")
    print(e)

Table created successfully.


In [4]:
import json

import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('quick-photos')

items = []

with open('../scripts/items.json', 'r') as f:
    for row in f:
        items.append(json.loads(row))

with table.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)

In [6]:
import boto3

from entities import User, Photo

dynamodb = boto3.client('dynamodb')

USER = "jacksonjason"


def fetch_user_and_photos(username):
    resp = dynamodb.query(
        TableName='quick-photos',
        KeyConditionExpression="PK = :pk AND SK BETWEEN :metadata AND :photos",
        ExpressionAttributeValues={
            ":pk": { "S": "USER#{}".format(username) },
            ":metadata": { "S": "#METADATA#{}".format(username) },
            ":photos": { "S": "PHOTO$" },
        },
        ScanIndexForward=True
    )

    user = User(resp['Items'][0])
    user.photos = [Photo(item) for item in resp['Items'][1:]]

    return user


user = fetch_user_and_photos(USER)

print(user)
for photo in user.photos:
    print(photo)

User<jacksonjason -- John Perry>
Photo<jacksonjason -- 2018-05-30T15:42:38>
Photo<jacksonjason -- 2018-06-09T13:49:13>
Photo<jacksonjason -- 2018-06-26T03:59:33>
Photo<jacksonjason -- 2018-07-14T10:21:01>
Photo<jacksonjason -- 2018-10-06T22:29:39>
Photo<jacksonjason -- 2018-11-13T08:23:00>
Photo<jacksonjason -- 2018-11-18T15:37:05>
Photo<jacksonjason -- 2018-11-26T22:27:44>
Photo<jacksonjason -- 2019-01-02T05:09:04>
Photo<jacksonjason -- 2019-01-23T12:43:33>
Photo<jacksonjason -- 2019-03-03T02:00:01>
Photo<jacksonjason -- 2019-03-03T18:20:10>
Photo<jacksonjason -- 2019-03-11T15:18:22>
Photo<jacksonjason -- 2019-03-30T02:28:42>
Photo<jacksonjason -- 2019-04-14T21:52:36>
